In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("spark_dataframe_api")
    .getOrCreate()
)

24/05/13 01:48:45 WARN Utils: Your hostname, Home-PC resolves to a loopback address: 127.0.1.1; using 172.21.161.222 instead (on interface eth0)
24/05/13 01:48:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/13 01:48:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/13 01:48:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-04.csv')
)

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
df_precos = (
    df
    .select("Estado - Sigla", "Produto", "Valor de Venda", "Unidade de Medida")
)

In [6]:
df_precos.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            SP|          GASOLINA|          4,88|       R$ / litro|
|            SP|            ETANOL|          3,09|       R$ / litro|
|            RS|          GASOLINA|          5,49|       R$ / litro|
|            RS|GASOLINA ADITIVADA|          5,49|       R$ / litro|
|            RS|            ETANOL|          4,39|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [7]:
df_precos = (
    df
    .select("Estado - Sigla", "Produto", "Valor de Venda", "Unidade de Medida")
    .withColumn(
        "Valor de Venda",
        F.regexp_replace(F.col("Valor de Venda"), ",", ".")
        .cast("float")
    )
)

In [8]:
df_precos_analise = (
    df_precos
    .groupBy(
        F.col('Estado - Sigla'),
        F.col('Produto'),
        F.col('Unidade de Medida')
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias('menor_valor'),
        F.max(F.col("Valor de Venda")).alias('maior_valor')
    )
    .withColumn(
        "diferenca",
        F.col('menor_valor') - F.col('maior_valor')
    )
    .orderBy('diferenca', ascending=False)
)


In [9]:
df_precos_analise.show()

+--------------+------------------+-----------------+-----------+-----------+------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|   diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------+
|            RR|          GASOLINA|       R$ / litro|       6.15|       6.19| -0.03999996|
|            AP|            ETANOL|       R$ / litro|       4.94|       4.99|-0.049999714|
|            RR|            ETANOL|       R$ / litro|       4.79|       4.87| -0.07999992|
|            RR|GASOLINA ADITIVADA|       R$ / litro|       6.15|       6.37| -0.21999979|
|            AP|GASOLINA ADITIVADA|       R$ / litro|       5.82|       6.15| -0.32999992|
|            AP|          GASOLINA|       R$ / litro|       5.45|       5.99| -0.53999996|
|            PI|          GASOLINA|       R$ / litro|       5.34|       5.95| -0.60999966|
|            DF|          GASOLINA|       R$ / litro|       5.36|       5.99| -0.62999964|